In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import holodeck.detstats as ds
import h5py

import numpy as np
import matplotlib.pyplot as plt

# Calculate Library Detstats

In [ ]:
# hdf_name = '/Users/emigardiner/GWs/holodeck/output/2023-05-09-mbp-ss14_n40_r10_f20_d17.5_l5_p0/ss_lib.hdf5'
# detstats_path = '/Users/emigardiner/GWs/holodeck/output/2023-05-09-mbp-ss14_n40_r10_f20_d17.5_l5_p0/detstats'

# npsrs = 40
# sigma = 1e-7
# output_dir = detstats_path+'/ds2_psrs40_sigma1e-7'

In [ ]:
# vals = ds.detect_lib(hdf_name, output_dir, npsrs, sigma, 
#                      nskies=25, thresh=0.5, plot=True, debug=True)

# Read in Library Detstats

In [ ]:
sspath = '/Users/emigardiner/GWs/holodeck/output/2023-05-09-mbp-ss15_n100_r30_f100_d15_l5_p0/'
hdfname = sspath+'ss_lib.hdf5'
ssfile = h5py.File(hdfname, 'r')
print(list(ssfile.keys()))
hc_ss = ssfile['hc_ss'][...]
hc_bg = ssfile['hc_bg'][...]
ssfile.close()

print('N,F,R,L:', hc_ss.shape) 
shape = hc_ss.shape
N, F, R, L = shape[0], shape[1], shape[2], shape[3]

# dsfile = np.load('/Users/emigardiner/GWs/holodeck/output/2023-05-09-mbp-ss15_n100_r30_f100_d15_l5_p0/detstats/dp4_psrs60_sigma1e-07/detstats.npz')
dsfile = np.load('/Users/emigardiner/GWs/holodeck/output/2023-05-12-mbp-ss16_n10_r10_f70_d12.5_l10_p0/detstats/ds2_psrs60_sigma5e-07/detstats.npz')
dp_ss = dsfile['dp_ss']
dp_bg = dsfile['dp_bg']
df_ss = dsfile['df_ss']
df_bg = dsfile['df_bg']
snr_ss = dsfile['snr_ss']
snr_bg = dsfile['snr_bg']
print('N,R,S:', dp_ss.shape)
S = dp_ss.shape[-1]

In [ ]:
print('snr_ss range:', np.min(snr_ss), np.max(snr_ss))

# Integration Outputs

In [ ]:
from scipy import integrate, special
from sympy import nsolve, Symbol

def _Fe_thresh(Num, alpha_0=0.001):
    """ Calculate the threshold F_e statistic using sympy.nsolve
    
    Parameters
    ----------
    Num : int
        Number of single sources to detect.
    alpha_0 : scalar
        False alarm probability max.

    Returns
    -------
    Fe_bar : scalar
        Threshold Fe statistic
    """
    Fe_bar = Symbol('Fe_bar')
    func = 1 - (1 - (1 + Fe_bar)*np.e**(-Fe_bar))**Num - alpha_0
    Fe_bar = nsolve(func, Fe_bar, 10)
    return(Fe_bar)

def _integrand(Fe, rho):

    I_1 = special.i1(rho*np.sqrt(2*Fe))
    rv = (2*Fe)**(1/2) /rho * I_1 * np.exp(-Fe - rho**2 /2)
    return rv

def integrate_rho(Fe_bar, rho):
    """ Calculate the detection probability for each single source in each realization.
    
    Parameters
    ----------
    rho : scalar
    Fe_bar : scalar

    Returns
    -------
    rv : float
    """
    gamma_ssi = integrate.quad(_integrand, Fe_bar, np.inf, 
                               args=(rho))[0]
    return gamma_ssi

In [ ]:
Num = F*L
Fe_bar = _Fe_thresh(Num)
outname = './output/integral_maps/Num%d_F%d_L%d_Febar%.2f.dat' % (Num, F, L, Fe_bar)

In [ ]:
rho = np.geomspace(10**-3, 10**3, 10**5)
print(rho.shape)
ans = np.zeros_like(rho)


for ii in range(len(rho)):
    if ii%(len(rho)/10) == 0:
       print(ii, 'out of', len(rho))
    ans[ii] = integrate_rho(Fe_bar, rho[ii])


In [ ]:
xx = rho
yy = ans

fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(8,5),
                               gridspec_kw={'height_ratios': [3.5, 1]})
ax1.loglog(xx, yy, label='Num=%d, Fe_bar=%.2f' % (Num, Fe_bar), marker='x', markersize=5, linestyle='')
ax1.set_title(r'$\gamma_i(\rho) = \int_{\bar{\mathcal{F}}_e}^\infty \frac{\sqrt{2\mathcal{F}_e}}{\rho} \mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) e^{-\mathcal{F}_e - \frac{1}{2}\rho^2} d\mathcal{F}_e$', 
               fontsize=12)
ax1.set_ylabel('$\gamma_i$')
ax1.axhline(1, color='k', alpha=0.5, ls='--', label='$\gamma_i$=1')
ax1.legend()

ax2.scatter(xx, np.isnan(yy), marker='x', s=5, color='tab:red')
ax2.set_ylabel('is nan')
ax2.set_xlabel(r'$\rho$', fontsize=14)

fig.tight_layout()

In [ ]:
rho1 = np.geomspace(10**0.5, 10**2, 10**3)
print(rho1.shape)
ans1 = np.zeros_like(rho1)


for ii in range(len(rho1)):
    if ii%(len(rho1)/10) == 0:
       print('on', ii, 'out of', len(rho1))
    ans1[ii] = integrate_rho(Fe_bar, rho1[ii])

In [ ]:
xx=rho1
yy=ans1
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(8,5),
                               gridspec_kw={'height_ratios': [3.5, 1]})
ax1.loglog(xx, yy, label='Num=%d, Fe_bar=%.2f' % (Num, Fe_bar), marker='x', markersize=5, linestyle='')
ax1.set_title(r'$\gamma_i(\rho) = \int_{\bar{\mathcal{F}}_e}^\infty \frac{\sqrt{2\mathcal{F}_e}}{\rho} \mathcal{I}_1 ( \rho\sqrt{2\mathcal{F}_e} ) e^{-\mathcal{F}_e - \frac{1}{2}\rho^2} d\mathcal{F}_e$', 
               fontsize=12)
ax1.set_ylabel('$\gamma_i$')
ax1.axhline(1, color='k', alpha=0.5, ls='--', label='$\gamma_i$=1')
ax1.legend()

ax2.scatter(xx, np.isnan(yy), marker='x', s=5, color='tab:red')
ax2.set_ylabel('is nan')
ax2.set_xlabel(r'$\rho$', fontsize=14)

fig.tight_layout()